https://www.trulens.org/trulens_eval/getting_started/quickstarts/llama_index_quickstart/

In [1]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [2]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.chunk_size = 128
Settings.chunk_overlap = 16

Settings.llm = OpenAILike(
    model="qwen2", 
    api_base="http://monkey:11434/v1", 
    api_key="ollama",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model =OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 26.5 ms, sys: 0 ns, total: 26.5 ms
Wall time: 27.3 ms


In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

Settings.chunk_size = 128
Settings.chunk_overlap = 16

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(similarity_top_k=3)

In [4]:
response = query_engine.query("北海公园在哪里？")
print(response)

北海公园位于北京市西城区文津街1号。


In [5]:
import os
from trulens_eval.feedback.provider import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"
os.environ["OPENAI_API_BASE"] = "https://ape:3000/v1"
provider = OpenAI(
    model_engine="qwen2"
)

In [6]:
from trulens_eval import Feedback
import numpy as np

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(query_engine)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(context.collect()) # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [7]:
from trulens_eval import TruLlama
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance])

In [8]:
import nltk
nltk.set_proxy('http://myproxy:7890')

# or as context manager
with tru_query_engine_recorder as recording:
    query_engine.query("北海公园在哪里？")

In [9]:
records, feedback = tru.get_records_and_feedback(
    app_ids=[]
)  # pass an empty list of app_ids to get all
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_2bdf00b5dc358cfeadc8e0340a395ef4,"""\u5317\u6d77\u516c\u56ed\u5728\u54ea\u91cc\uf...","""\u5317\u6d77\u516c\u56ed\u4f4d\u4e8e\u5317\u4...",-,"{""record_id"": ""record_hash_2bdf00b5dc358cfeadc...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-05T20:15:51.971980"", ""...",2024-08-05T20:15:55.638106,3,437,0.0


In [10]:
import nltk
nltk.set_proxy('http://myproxy:7890')

# or as context manager
with tru_query_engine_recorder as recording:
    query_engine.query("北海公园是啥时候建立的？")

In [11]:
records, feedback = tru.get_records_and_feedback(
    app_ids=[]
)  # pass an empty list of app_ids to get all
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_2bdf00b5dc358cfeadc8e0340a395ef4,"""\u5317\u6d77\u516c\u56ed\u5728\u54ea\u91cc\uf...","""\u5317\u6d77\u516c\u56ed\u4f4d\u4e8e\u5317\u4...",-,"{""record_id"": ""record_hash_2bdf00b5dc358cfeadc...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-05T20:15:51.971980"", ""...",2024-08-05T20:15:55.638106,3,437,0.0
1,LlamaIndex_App1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_e58e183336feddcd7651e3acee925617,"""\u5317\u6d77\u516c\u56ed\u662f\u5565\u65f6\u5...","""\u5317\u6d77\u516c\u56ed\u662f\u5728\u8f9b\u4...",-,"{""record_id"": ""record_hash_e58e183336feddcd765...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-08-05T20:16:44.985544"", ""...",2024-08-05T20:16:47.190832,2,396,0.0


In [14]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, "Context Relevance")

""


In [15]:
from trulens_eval.guardrails.llama import WithFeedbackFilterNodes

# note: feedback function used for guardrail must only return a score, not also reasons
f_context_relevance_score = Feedback(provider.context_relevance)

filtered_query_engine = WithFeedbackFilterNodes(
    query_engine, 
    feedback=f_context_relevance_score, 
    threshold=0.5)

In [18]:
tru_recorder = TruLlama(filtered_query_engine,
    app_id='LlamaIndex_App1_Filtered',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

with tru_recorder as recording:
    llm_response = filtered_query_engine.query("北海公园是啥时候建立的？")

# display(llm_response)

RuntimeError: Endpoint openai request failed 4 time(s): 
	Connection error.
	Connection error.
	Connection error.
	Connection error.